In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os.path
from pandas_ods_reader import read_ods

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers

import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

%matplotlib inline

Using TensorFlow backend.


In [3]:
fighter_sheet = "raw_fighter_details"
fight_sheet = "raw_total_fight_data"

fighters_ods = "UFC-DATA-DELETE-LATER/raw_fighter_details.ods"
fights_ods = "UFC-DATA-DELETE-LATER/raw_total_fight_data.ods"

In [4]:
fights_data = read_ods(fights_ods, fight_sheet)
fighters_data = read_ods(fighters_ods, fighter_sheet)

In [5]:
fighters_data.head(50)

,fighter_name,Height,Weight,Reach,Stance,DOB
0,AJ Fonseca,"5' 4""",145 lbs.,None,None,None
1,AJ Matthews,"5' 11""",185 lbs.,None,None,None
2,AJ McKee,"5' 10""",145 lbs.,None,None,None
3,AJ Siscoe,"5' 7""",135 lbs.,None,None,None
4,Aalon Cruz,"6' 0""",145 lbs.,None,None,None
5,Aaron Brink,"6' 3""",205 lbs.,None,Orthodox,"Nov 12, 1974"
6,Aaron Ely,"5' 8""",135 lbs.,None,None,"Mar 18, 1989"
7,Aaron Jeffery,"6' 2""",185 lbs.,None,None,"Nov 14, 1992"
8,Aaron Lanfranco,None,155 lbs.,None,None,"Aug 26, 1986"
9,Aaron Miller,"5' 9""",145 lbs.,None,None,None


In [6]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

In [7]:
fights_data['R_SIG_STR_pct'] = fights_data['R_SIG_STR_pct'].str.strip('%').astype(float)
fights_data['B_SIG_STR_pct'] = fights_data['B_SIG_STR_pct'].str.strip('%').astype(float)
fights_data['R_TD_pct'] = fights_data['R_TD_pct'].str.strip('%').astype(float)
fights_data['B_TD_pct'] = fights_data['B_TD_pct'].str.strip('%').astype(float)

In [8]:
fights_data['R_SIG_STR_pct'] = fights_data['R_SIG_STR_pct']/100
fights_data['B_SIG_STR_pct'] = fights_data['B_SIG_STR_pct']/100
fights_data['R_TD_pct'] = fights_data['R_TD_pct']/100
fights_data['B_TD_pct'] = fights_data['B_TD_pct']/100


In [21]:
newfights_data = fights_data.drop(["R_TOTAL_STR.", "B_TOTAL_STR.", "R_TD", "B_TD"], axis=1)
newfights_data

,R_fighter,B_fighter,R_KD,B_KD,R_SIG_STR.,B_SIG_STR.,R_SIG_STR_pct,B_SIG_STR_pct,R_TD_pct,B_TD_pct,R_SUB_ATT,B_SUB_ATT,R_PASS,B_PASS,R_REV,B_REV,R_HEAD,B_HEAD,R_BODY,B_BODY,R_LEG,B_LEG,R_DISTANCE,B_DISTANCE,R_CLINCH,B_CLINCH,R_GROUND,B_GROUND,win_by,last_round,last_round_time,Format,Referee,date,location,Fight_type,Winner
0,Henry Cejudo,Marlon Moraes,0.0,0.0,90 of 171,57 of 119,0.52,0.47,0.25,0.00,1.0,0.0,1.0,0.0,0.0,0.0,73 of 150,35 of 89,13 of 16,7 of 8,4 of 5,15 of 22,45 of 118,54 of 116,19 of 23,2 of 2,26 of 30,1 of 1,KO/TKO,3.0,4:51,5 Rnd (5-5-5-5-5),Marc Goddard,"June 08, 2019","Chicago, Illinois, USA",UFC Bantamweight Title Bout,Henry Cejudo
1,Valentina Shevchenko,Jessica Eye,1.0,0.0,8 of 11,2 of 12,0.72,0.16,1.00,0.00,1.0,0.0,3.0,0.0,0.0,0.0,4 of 5,0 of 7,4 of 6,0 of 2,0 of 0,2 of 3,5 of 8,2 of 12,2 of 2,0 of 0,1 of 1,0 of 0,KO/TKO,2.0,0:26,5 Rnd (5-5-5-5-5),Robert Madrigal,"June 08, 2019","Chicago, Illinois, USA",UFC Women's Flyweight Title Bout,Valentina Shevchenko
2,Tony Ferguson,Donald Cerrone,0.0,0.0,104 of 200,68 of 185,0.52,0.36,0.00,1.00,0.0,0.0,0.0,0.0,0.0,0.0,65 of 144,43 of 152,25 of 37,15 of 23,14 of 19,10 of 10,103 of 198,68 of 184,1 of 2,0 of 1,0 of 0,0 of 0,TKO - Doctor's Stoppage,2.0,5:00,3 Rnd (5-5-5),Dan Miragliotta,"June 08, 2019","Chicago, Illinois, USA",Lightweight Bout,Tony Ferguson
3,Jimmie Rivera,Petr Yan,0.0,2.0,73 of 192,56 of 189,0.38,0.29,0.00,0.33,0.0,0.0,0.0,1.0,0.0,0.0,42 of 145,40 of 166,15 of 24,13 of 19,16 of 23,3 of 4,60 of 173,42 of 167,9 of 15,10 of 12,4 of 4,4 of 10,Decision - Unanimous,3.0,5:00,3 Rnd (5-5-5),Kevin MacDonald,"June 08, 2019","Chicago, Illinois, USA",Bantamweight Bout,Petr Yan
4,Tai Tuivasa,Blagoy Ivanov,0.0,1.0,64 of 144,73 of 123,0.44,0.59,0.00,1.00,0.0,0.0,0.0,0.0,0.0,0.0,39 of 114,65 of 114,6 of 7,7 of 8,19 of 23,1 of 1,50 of 126,62 of 111,14 of 18,5 of 6,0 of 0,6 of 6,Decision - Unanimous,3.0,5:00,3 Rnd (5-5-5),Dan Miragliotta,"June 08, 2019","Chicago, Illinois, USA",Heavyweight Bout,Blagoy Ivanov
5,Tatiana Suarez,Nina Ansaroff,0.0,0.0,75 of 142,48 of 99,0.52,0.48,0.30,0.00,0.0,0.0,2.0,0.0,0.0,0.0,29 of 85,31 of 66,22 of 29,15 of 26,24 of 28,2 of 7,42 of 100,39 of 89,7 of 7,4 of 5,26 of 35,5 of 5,Decision - Unanimous,3.0,5:00,3 Rnd (5-5-5),Robert Madrigal,"June 08, 2019","Chicago, Illinois, USA",Women's Strawweight Bout,Tatiana Suarez
6,Aljamain Sterling,Pedro Munhoz,0.0,0.0,174 of 349,105 of 265,0.49,0.39,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,123 of 250,44 of 159,36 of 73,29 of 64,15 of 26,32 of 42,172 of 338,102 of 257,2 of 11,3 of 8,0 of 0,0 of 0,Decision - Unanimous,3.0,5:00,3 Rnd (5-5-5),Marc Goddard,"June 08, 2019","Chicago, Illinois, USA",Bantamweight Bout,Aljamain Sterling
7,Karolina Kowalkiewicz,Alexa Grasso,0.0,0.0,90 of 232,148 of 369,0.38,0.40,0.00,0.00,0.0,1.0,0.0,0.0,0.0,0.0,55 of 186,92 of 289,27 of 33,33 of 46,8 of 13,23 of 34,79 of 217,123 of 339,11 of 15,25 of 30,0 of 0,0 of 0,Decision - Unanimous,3.0,5:00,3 Rnd (5-5-5),Kevin MacDonald,"June 08, 2019","Chicago, Illinois, USA",Women's Strawweight Bout,Alexa Grasso
8,Ricardo Lamas,Calvin Kattar,0.0,1.0,12 of 29,22 of 41,0.41,0.53,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,2 of 12,11 of 28,4 of 5,11 of 13,6 of 12,0 of 0,12 of 29,16 of 34,0 of 0,2 of 2,0 of 0,4 of 5,KO/TKO,1.0,4:06,3 Rnd (5-5-5),Dan Miragliotta,"June 08, 2019","Chicago, Illinois, USA",Featherweight Bout,Calvin Kattar
9,Yan Xiaonan,Angela Hill,0.0,0.0,94 of 249,71 of 144,0.37,0.49,1.00,0.33,0.0,1.0,1.0,0.0,0.0,0.0,64 of 199,44 of 110,11 of 22,21 of 26,19 of 28,6 of 8,83 of 229,56 of 127,8 of 17,13 of 15,3 of 3,2 of 2,Decision - Unanimous,3.0,5:00,3 Rnd (5-5-5),Robert Madrigal,"June 08, 2019","Chicago, Illinois, USA",Women's Strawweight Bout,Yan Xiaonan


In [62]:
significant = newfights_data[['R_HEAD','B_HEAD','R_BODY','B_BODY','R_LEG','B_LEG','R_CLINCH','B_CLINCH','R_GROUND','B_GROUND']]
sig_col = significant.columns
sig_df = pd.DataFrame()

for col in sig_col:
    sig_series = significant[col].str.split(' of ', expand = True)
    sig_df[col] = sig_series[0].astype(float)/sig_series[1].astype(float)

sig_df.fillna(0)

,R_HEAD,B_HEAD,R_BODY,B_BODY,R_LEG,B_LEG,R_CLINCH,B_CLINCH,R_GROUND,B_GROUND
0,0.486667,0.393258,0.812500,0.875000,0.800000,0.681818,0.826087,1.000000,0.866667,1.000000
1,0.800000,0.000000,0.666667,0.000000,0.000000,0.666667,1.000000,0.000000,1.000000,0.000000
2,0.451389,0.282895,0.675676,0.652174,0.736842,1.000000,0.500000,0.000000,0.000000,0.000000
3,0.289655,0.240964,0.625000,0.684211,0.695652,0.750000,0.600000,0.833333,1.000000,0.400000
4,0.342105,0.570175,0.857143,0.875000,0.826087,1.000000,0.777778,0.833333,0.000000,1.000000
5,0.341176,0.469697,0.758621,0.576923,0.857143,0.285714,1.000000,0.800000,0.742857,1.000000
6,0.492000,0.276730,0.493151,0.453125,0.576923,0.761905,0.181818,0.375000,0.000000,0.000000
7,0.295699,0.318339,0.818182,0.717391,0.615385,0.676471,0.733333,0.833333,0.000000,0.000000
8,0.166667,0.392857,0.800000,0.846154,0.500000,0.000000,0.000000,1.000000,0.000000,0.800000
9,0.321608,0.400000,0.500000,0.807692,0.678571,0.750000,0.470588,0.866667,1.000000,1.000000


In [10]:
temp_df1 = fights_data['R_TOTAL_STR.'].str.split(' of ', expand = True)
temp_df1['R_TOTAL_STR.'] = temp_df1[0].astype(float)/temp_df1[1].astype(float)

temp_df2 = fights_data['B_TOTAL_STR.'].str.split(' of ', expand = True)
temp_df2['B_TOTAL_STR.'] = temp_df2[0].astype(float)/temp_df2[1].astype(float)

temp_df3 = fights_data['R_TD'].str.split(' of ', expand = True)
temp_df3['R_TD'] = temp_df3[0].astype(float)/temp_df3[1].astype(float)

temp_df4 = fights_data['B_TD'].str.split(' of ', expand = True)
temp_df4['B_TD'] = temp_df3[0].astype(float)/temp_df3[1].astype(float)

In [11]:
newtemp_df1 = temp_df1.drop([0, 1], axis=1)
newtemp_df2 = temp_df2.drop([0, 1], axis=1)
newtemp_df3 = temp_df3.drop([0, 1], axis=1)
newtemp_df4 = temp_df4.drop([0, 1], axis=1)

,R_fighter,B_fighter,R_KD,B_KD,R_SIG_STR.,B_SIG_STR.,R_SIG_STR_pct,B_SIG_STR_pct,R_TD_pct,B_TD_pct,R_SUB_ATT,B_SUB_ATT,R_PASS,B_PASS,R_REV,B_REV,R_HEAD,B_HEAD,R_BODY,B_BODY,R_LEG,B_LEG,R_DISTANCE,B_DISTANCE,R_CLINCH,B_CLINCH,R_GROUND,B_GROUND,win_by,last_round,last_round_time,Format,Referee,date,location,Fight_type,Winner,R_TOTAL_STR.,B_TOTAL_STR.,R_TD,B_TD
0,Henry Cejudo,Marlon Moraes,0.0,0.0,90 of 171,57 of 119,0.52,0.47,0.25,0.00,1.0,0.0,1.0,0.0,0.0,0.0,73 of 150,35 of 89,13 of 16,7 of 8,4 of 5,15 of 22,45 of 118,54 of 116,19 of 23,2 of 2,26 of 30,1 of 1,KO/TKO,3.0,4:51,5 Rnd (5-5-5-5-5),Marc Goddard,"June 08, 2019","Chicago, Illinois, USA",UFC Bantamweight Title Bout,Henry Cejudo,0.543956,0.487603,0.25,0.25
1,Valentina Shevchenko,Jessica Eye,1.0,0.0,8 of 11,2 of 12,0.72,0.16,1.00,0.00,1.0,0.0,3.0,0.0,0.0,0.0,4 of 5,0 of 7,4 of 6,0 of 2,0 of 0,2 of 3,5 of 8,2 of 12,2 of 2,0 of 0,1 of 1,0 of 0,KO/TKO,2.0,0:26,5 Rnd (5-5-5-5-5),Robert Madrigal,"June 08, 2019","Chicago, Illinois, USA",UFC Women's Flyweight Title Bout,Valentina Shevchenko,0.925000,0.807692,1.00,1.00
2,Tony Ferguson,Donald Cerrone,0.0,0.0,104 of 200,68 of 185,0.52,0.36,0.00,1.00,0.0,0.0,0.0,0.0,0.0,0.0,65 of 144,43 of 152,25 of 37,15 of 23,14 of 19,10 of 10,103 of 198,68 of 184,1 of 2,0 of 1,0 of 0,0 of 0,TKO - Doctor's Stoppage,2.0,5:00,3 Rnd (5-5-5),Dan Miragliotta,"June 08, 2019","Chicago, Illinois, USA",Lightweight Bout,Tony Ferguson,0.520000,0.367568,NaN,NaN
3,Jimmie Rivera,Petr Yan,0.0,2.0,73 of 192,56 of 189,0.38,0.29,0.00,0.33,0.0,0.0,0.0,1.0,0.0,0.0,42 of 145,40 of 166,15 of 24,13 of 19,16 of 23,3 of 4,60 of 173,42 of 167,9 of 15,10 of 12,4 of 4,4 of 10,Decision - Unanimous,3.0,5:00,3 Rnd (5-5-5),Kevin MacDonald,"June 08, 2019","Chicago, Illinois, USA",Bantamweight Bout,Petr Yan,0.389744,0.302083,0.00,0.00
4,Tai Tuivasa,Blagoy Ivanov,0.0,1.0,64 of 144,73 of 123,0.44,0.59,0.00,1.00,0.0,0.0,0.0,0.0,0.0,0.0,39 of 114,65 of 114,6 of 7,7 of 8,19 of 23,1 of 1,50 of 126,62 of 111,14 of 18,5 of 6,0 of 0,6 of 6,Decision - Unanimous,3.0,5:00,3 Rnd (5-5-5),Dan Miragliotta,"June 08, 2019","Chicago, Illinois, USA",Heavyweight Bout,Blagoy Ivanov,0.452055,0.618321,NaN,NaN


In [77]:
df72['new_column'] = np.where(df72['R_fighter'] == df72['Winner'], '0', '1')
df72.new_column.head()

0    0
1    0
3    1
5    0
6    0
Name: new_column, dtype: object

In [85]:
df72 = df72.fillna(0)
df72 = df72.drop(['R_HEAD','B_HEAD','R_BODY','B_BODY','R_LEG','B_LEG','R_CLINCH','B_CLINCH','R_GROUND','B_GROUND'],axis=1)
df72 = df72

In [79]:
# Creating input feature
X = df72[['R_KD','B_KD','R_SIG_STR_pct', 'B_SIG_STR_pct', 'R_TOTAL_STR.', 'B_TOTAL_STR.',
                 'R_TD','B_TD','R_TD_pct','B_TD_pct','R_SUB_ATT','B_SUB_ATT','R_PASS','B_PASS','R_REV','B_REV']]

# Creating target variable
y = df72[['new_column']]

#Creating Train and Test dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# #Normalizing the input features
# from sklearn.preprocessing import StandardScaler
# sc_x = StandardScaler()
# X_train = sc_x.fit_transform(X_train)
# X_test = sc_x.transform(X_test)

# #Normalizing the target variable
# sc_y = StandardScaler()
# y_train = sc_y.fit_transform(y_train)
# y_test = sc_y.transform(y_test)

# Creating the Logistic regressor
classifier = LogisticRegression()

# fitting the training data to the Logistic Regressor
classifier.fit(X_train,y_train)

# Checking the model coefficients
classifier.coef_

C:\Users\Public\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Public\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[-1.11287144,  1.24341399, -1.33557832,  3.88131251, -2.88148462,
         0.2604436 , -0.31622585, -0.31622585, -0.35194395,  0.66758787,
        -0.47771993,  0.15994343, -0.28608604,  0.39664054, -0.61035558,
         0.4505602 ]])

In [80]:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
                   solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [81]:
classifier.fit(X_train, y_train)
test_results = classifier.predict(X_test)
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.8267552970502701
Testing Data Score: 0.8343023255813954


C:\Users\Public\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Public\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [82]:
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].values.tolist()}")

First 10 Predictions:   ['0' '0' '1' '0' '0' '0' '0' '0' '0' '0']
First 10 Actual labels: [['0'], ['0'], ['1'], ['0'], ['0'], ['0'], ['0'], ['0'], ['1'], ['0']]


In [83]:
accuracy = classifier.score(X_test, y_test)
print('The accuracy is: ' + str(accuracy *100) + '%')

The accuracy is: 83.43023255813954%


In [84]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test['new_column']}).reset_index(drop=True)

,Prediction,Actual
0,0,0
1,0,0
2,1,1
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,1
9,0,0
